In [ ]:
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import altair as alt
from altair import datum, expr
from vega_datasets import data

alt.data_transformers.disable_max_rows()

In [ ]:
def build_dataframe(path='messages'):
    rows_list = []
    for filename in Path(path).glob('inbox/*/message_*.json'):
        chat = filename.parent.name
        with open(filename, 'r') as f:
            obj = json.load(f)
        for entry in obj['messages']:
            if entry['type'] == 'Generic' and entry.get('content') is not None:
                rows_list.append({
                    'chat': chat,
                    'sender': entry['sender_name'],
                    'time': pd.to_datetime(entry['timestamp_ms'], unit='ms'),
                    'content': entry['content'],
                })
    return pd.DataFrame(rows_list)

df = build_dataframe()

In [ ]:
df

In [ ]:
df.chat.value_counts()

In [ ]:
alt.Chart(df).mark_line().encode(
    x='yearmonth(time):T',
    y='count()',
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='yearmonth(time):T',
    y='count()',
    color='chat:N',
    tooltip=['chat', 'count()'],
).properties(
    title='Number of Facebook Messages',
).interactive()

In [ ]:
chat_id = '21buddies__yjaob9r8g'

alt.Chart(df.query(f'chat == "{chat_id}"')).mark_bar().encode(
    color='sender:N',
    x='yearmonth(time):T',
    y='count()',
    tooltip=['sender', 'count()'],
).properties(
    title='Facebook Messages in Group Chat',
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('count()', stack='normalize', title='frequency'),
    alt.Y('chat'),
    alt.Color('sender'),
    tooltip=['sender', alt.Tooltip('count()', title='messages')],
).properties(
    title='Who Dominates the Conversation?',
)

In [ ]:
sender = 'Cindy Liu'

alt.Chart(df).mark_rect().encode(
    alt.X('date(time):O', title='day'),
    alt.Y('yearmonth(time):O', title='month'),
    alt.Color('count()', scale=alt.Scale(type='linear')),
    tooltip=[
        alt.Tooltip('count()', title='Messages'),
        alt.Tooltip('sum(words):Q', title='Words'),
    ],
).transform_filter(
    datum.sender == sender,
).transform_calculate(
    words=expr.length(expr.split(datum.content, ' ')),
).properties(
    title='Number of Messages Sent by Day',
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    
    x=alt.X("sender:N", title="Sender", sort = alt.EncodingSortField(
                    field="words", op="mean", order="descending")),
    y=alt.Y("average(words):Q", title="Average Length of Message"),
    color=alt.Color('average(words):Q', scale=alt.Scale(scheme='teals')),
    tooltip=[
        alt.Tooltip('count()', title='Messages'),
        alt.Tooltip('average(words):Q', title='Average Words'),
    ],
).transform_calculate(
    words=expr.length(expr.split(datum.content, ' ')),
).properties(
    title='Average Length of Texts',
)

In [ ]:
me = 'Cindy Liu'

alt.Chart(df).mark_rect().encode(
    alt.X('date(time):O', title='day'),
    alt.Y('yearmonth(time):O', title='month'),
    alt.Color('count()', scale=alt.Scale(scheme='purplered')),
    tooltip=[
        alt.Tooltip('count()', title='Messages'),
        alt.Tooltip('sum(words):Q', title='Words'),
    ],
).transform_filter(
    datum.sender != me,
).transform_calculate(
    words=expr.length(expr.split(datum.content, ' ')),
).properties(
    title='Number of Messages Received By Day',
)